### 19BIO211 INTELLIGENCE IN BIOLOGICAL SYSTEMS FINAL PROJECT PYTHON FILE
#### AUTOENCODERS BASED APPROACH FOR BIO SEQUENCE CLASSIFICATION
#### TEAM 14

###### *TEAM MEMBERS*

###### C SATYANANTH    [CB.EN.U4AIE19019]
###### DARSHAN D       [CB.EN.U4AIE19021]
###### M GNANADEEP     [CB.EN.U4AIE19041]
###### TARUN SANJEEV S [CB.EN.U4AIE19064]

In [ ]:
# First we import the necessary libraries
import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility

import keras
import tensorflow as tf
from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.regularizers import l2
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Bidirectional

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
# Here we load the data from file and store in variables
print('loading data')
trainmat = h5py.File('F:\DEEP SEA TRAIN BUNDLE KERAS DATASET IBS SEM 4 PRJOECT\deepsea_train_bundle.v0.9\deepsea_train\train.mat')
validmat = scipy.io.loadmat('F:\DEEP SEA TRAIN BUNDLE KERAS DATASET IBS SEM 4 PRJOECT\deepsea_train_bundle.v0.9\deepsea_train\valid.mat')
testmat = scipy.io.loadmat('F:\DEEP SEA TRAIN BUNDLE KERAS DATASET IBS SEM 4 PRJOECT\deepsea_train_bundle.v0.9\deepsea_train\test.mat')

# Develop the test and train parts
X_train = np.transpose(np.array(trainmat['trainxdata']),axes=(2,0,1))
y_train = np.array(trainmat['traindata']).T

In [ ]:
# Defining the LSTM Type Autoencoder for sequence classification
forward_lstm = LSTM(units=320, return_sequences=True)
# backward_lstm = LSTM(input_dim=320, output_dim=320, return_sequences=True)
brnn = Bidirectional(forward_lstm)

print('building model')

# Creating the model and defining the necessary specifications
model = Sequential()
model.add(Convolution1D(activation="relu", 
                        input_shape=(1000, 4), 
                        padding="valid", strides=1, 
                        filters=320, kernel_size=26))

model.add(MaxPooling1D(strides=13, pool_size=13))
model.add(Dropout(0.2))
model.add(brnn)
model.add(Dropout(0.5))
model.add(Flatten())

model.add(Dense(input_dim=75*640, units=925))
model.add(Activation('relu'))

model.add(Dense(input_dim=925, units=919))
model.add(Activation('sigmoid'))

# Compiling the model
print('compiling model')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['cosine'])

print('running at most 60 epochs')

# Prevents rerun of the model when subjected to crashes. We can restart from last checkpoint
checkpointer = ModelCheckpoint(filepath="DanQ_bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# Fitting the model
model.fit(X_train, y_train, batch_size=2048, epochs=60, 
          shuffle=True,
          validation_data=(np.transpose(validmat['validxdata'],axes=(0,2,1)), validmat['validdata']), 
          callbacks=[checkpointer,earlystopper])

In [ ]:
# Extracting the final results
tresults = model.evaluate(np.transpose(testmat['testxdata'],axes=(0,2,1)), 
                          testmat['testdata'])

print tresults

In [ ]:
# Predictions
preds = model.predict(np.transpose(testmat['testxdata'],axes=(0,2,1)))
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

In [ ]:
# Final Encoding of the predicitons
preds_flat = preds.flatten()
test_flat = testmat['testdata'].flatten()

In [ ]:
# Printing the Final accuracy score and the confusion matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
print(accuracy_score(test_flat, preds_flat))
print(confusion_matrix(test_flat, preds_flat))